# Profiling TensorFlow Multi GPU Multi Node  Training Job with Amazon SageMaker Debugger

이 노트북은 SageMaker 디버거 프로파일링(debugger profiling) 기능이 활성화된 상태에서 TensorFlow 훈련 작업을 생성하는 과정을 안내합니다. Horovod를 사용하여 다중 GPU 다중 노드 훈련을 생성합니다.

### (Optional) Install SageMaker and SMDebug Python SDKs

2020년 12월에 출시된 새로운 디버거 프로파일링 기능을 사용하려면, 최신 버전의 SageMaker 및 SMDebug SDK가 설치되어 있는지 확인하세요. 다음 코드 셀을 사용하여 라이브러리를 업데이트하고 Jupyter 커널을 다시 시작하여 업데이트를 적용합니다.

In [ ]:
import sys
import IPython
install_needed = False  # should only be True once
if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U sagemaker smdebug
    IPython.Application.instance().kernel.do_shutdown(True)

## 1. Create a Training Job with Profiling Enabled<a class="anchor" id="option-1"></a>

[SageMaker Estimator API for Tensorflow](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator)를 사용하여 훈련 작업을 생성합니다. 프로파일링을 활성화하려면 `ProfilerConfig` 객체를 생성하고 TensorFlow estimator의 `profiler_config` 파라메터에 전달합니다.

### Define parameters for distributed training

이 파라메터는 SageMaker에 horovod를 구성하고 실행하는 방법을 알려줍니다. 노드당 4개 이상의 GPU를 사용하려면 그에 따라 `process_per_host` 파라메터를 변경해 주세요.

In [ ]:
distributions = {
    "mpi": {
        "enabled": True,
        "processes_per_host": 4,
        "custom_mpi_options": "-verbose -x HOROVOD_TIMELINE=./hvd_timeline.json -x NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none",
    }
}

### Configure rules

다음 규칙(rule)을 지정합니다.

- loss_not_decreasing : loss가 감소하고 있는지 확인하고 지난 몇 번의 iteration에서 loss이 특정 수치만큼 감소하지 않은 경우 트리거합니다.
- LowGPUUtilization : GPU의 활용도가 낮은지 확인합니다.
- ProfilerReport : 전체 퍼포먼스 규칙 세트를 실행하고 추가 정보 및 권장 사항이 포함된 최종 출력용 보고서를 생성합니다.

In [ ]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

### Specify a profiler configuration

다음 설정은 500 밀리초에서 시스템 메트릭을 캡처합니다. 시스템 지표에는 CPU 당 사용률, GPU, CPU 당 메모리 사용률, GPU, I/O 및 네트워크가 포함됩니다.

디버거는 5단계부터 15단계까지 상세한 프로파일링 정보를 캡처합니다. 이 정보에는 Horovod 메트릭, 데이터 로딩, 전처리, CPU 및 GPU에서 실행되는 연산자(operator)가 포함됩니다.

In [ ]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(
        local_path="/opt/ml/output/profiler/", start_step=5, num_steps=10
    ),
)

### Get the image URI

이 노트북을 실행하는 리전에 따라 도커 이미지가 달라집니다.

In [ ]:
import boto3

session = boto3.session.Session()
region = session.region_name

image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/tensorflow-training:2.3.1-gpu-py37-cu110-ubuntu18.04"

### Define estimator

프로파일링을 활성화하려면 디버거 프로파일링 구성 (`profiler_config`), 디버거 규칙 목록 (`rules`) 및 이미지 URI (`image_uri)`를 estimator에 전달해야 합니다. 디버거는 SageMaker estimator가 훈련 작업을 요청하는 동안 모니터링 및 프로파일링을 활성화합니다.

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    role=sagemaker.get_execution_role(),
    image_uri=image_uri,
    instance_count=2,
    instance_type="ml.p3.8xlarge",
    entry_point="tf-hvd-train.py",
    source_dir="entry_point",
    profiler_config=profiler_config,
    distribution=distributions,
    rules=rules,
)

### Start training job

`wait=False` argument를 포함한 `estimator.fit()`은 백그라운드에서 훈련 작업을 시작합니다. 대시보드 또는 분석 노트북 실행을 계속할 수 있습니다.

In [ ]:
estimator.fit(wait=False)

## 2.  Analyze Profiling Data

다음 셀 (`training_job_name` 및 `region`)의 출력을 복사하여 분석 노트북 `profiling_generic_dashboard.ipynb`, `analyze_performance_bottlenecks.ipynb` 및 `profiling_interactive_analysis.ipynb`를 실행합니다.

In [ ]:
training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

훈련이 아직 진행 중인 동안 SageMaker Studio 또는 노트북에서 성능 데이터를 시각화할 수 있습니다. 디버거는 타임라인 차트 또는 히트맵 형식으로 시스템 메트릭을 그리는 유틸리티를 제공합니다. 자세한 내용은 노트북 [profiling_interactive_analysis.ipynb](analysis_tools/profiling_interactive_analysis.ipynb) 을 확인하세요. 다음 코드 셀에서 총 CPU 및 GPU 사용률을 시계열 차트로 표시합니다. I/O, 메모리, 네트워크와 같은 다른 메트릭을 시각화하려면, `select_dimension` 및 `select_events`에 전달된 목록을 확장하기만 하면 됩니다.

### Install the SMDebug client library to use Debugger analysis tools

In [ ]:
import pip


def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(["install", package])


import_or_install("smdebug")

### Access the profiling data using the SMDebug `TrainingJob` utility class

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

### Plot time line charts

다음 코드는 SMDebug `TrainingJob` 객체를 사용하고, 새 이벤트 파일을 사용할 수 있는 경우 객체를 갱신하고, CPU 및 GPU 사용량의 타임라인 차트를 그리는 방법을 보여줍니다.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

## 3. Download Debugger Profiling Report

`ProfilerReport()` 규칙은 기본 규칙 요약 및 다음 단계 권장 사항이 포함된 html 리포트 `profiler-report.html`을 생성합니다. 이 리포트는 S3 버킷에서 찾을 수 있습니다.

In [ ]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

디버거 프로파일링 보고서를 다운로드하고 여는 방법에 대한 자세한 내용은 SageMaker 개발자 가이드의 [SageMaker Debugger Profiling Report](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-profiling-report.html)를 참조하세요.